# Bibliotecas Utilizadas

In [1]:
#Para manipulações de leitura, escrita e processamento de imagens
import cv2

#Para manipulações de imagens e operações matemáticas, geométricas e algébricas
import numpy as np

#Para os plots de resultados obtidos
import matplotlib.pyplot as plt

#Para binarização de imagens
#from skimage.filters import threshold_otsu

#Para as operações de processamento de sinais e imagens báscias
import scipy
from scipy.signal import convolve

#Para as operações de leitura e escrita de arquivos em diretórios
import os

#Para operações com parser dos arquivos XML no formato PASCAL VOC
import xml.etree.ElementTree as ET

#Para operações envolvendo números com muitas casas decimais
from decimal import *

# Definição de Funções Utilizadas

### - Funções para o pré-processamento das imagens

In [2]:
def unsharp_mask(img, intensity):
    '''
    np.array, float -> np.array
    Realça as bordas através da aplicação do unsharp mask 
    (convolução do filtro identidade - filtro laplaciano ponderado).
    
    Parâmetros
    ----------
    img : imagem de entrada a ser realçada pela técnica.
    intensity: intensidade da ponderação do realce de bordas.
    
    Return
    ------
    Numpy array contendo a imagem com bordas destacadas através do unsharp mask.
    '''
    kernel = np.array([[0,             -1*intensity,              0],
                       [-1*intensity,  (4*intensity)+1, -1*intensity],
                       [0,             -1*intensity,              0]])
    return convolve(img, kernel, mode='same')

### - Funções para a extração das anotações dos ground truths presentes nos arquivos XML.

In [3]:
def coord_to_center(xmin, ymin, xmax, ymax):
    '''
    int, int, int, int -> (int, int)
    Retorna as coordenadas do centro de uma anotação referente a um leucócito anotado como ground truth, a partir
    de dois pontos nos vértices opostos de um quadrado que o contenha.
    
    Parâmetros
    ----------
    xmin, ymin - coordenadas x e y do vértice superior esquerdo da anotação
    xmax, ymax - coordenadas x e y do vértice inferior direito da anotação
    
    Return
    ------
    coordenadas do ponto do centro da anotação
    '''
    
    return ((xmin+xmax)//2),((ymin+ymax)//2)

In [4]:
def read_content(xml_file):
    '''
    arquivo.xml -> string, list
    Realiza o parsing de um arquivo xml contendo as anotações ground truth de um quadro do vídeo
    e retorna o nome do quadro e a lista de coordenadas das caixas que contém as células anotadas.
    
    Parâmetros
    ----------
    xml_file: caminho para o arquivo xml
    
    Return
    ------
    filename: string com o nome do arquivo contendo o frame.
    list_with_all_boxes: literalmente uma lista contendo as coordenadas dos limites de todas as caixas 
    que contém as células anotadas naquele quadro.
    '''
    
    #Define a árvore, e a raiz da mesma, que estrutura o arquivo
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    #Inicializa uma lista vazia
    list_with_all_boxes = []
    list_with_all_centers = []

    #Faz a leitura e o parsing do arquivo
    for boxes in root.iter('object'):

        filename = root.find('filename').text

        ymin, xmin, ymax, xmax = None, None, None, None

        for box in boxes.findall("bndbox"):
            ymin = int(box.find("ymin").text)
            xmin = int(box.find("xmin").text)
            ymax = int(box.find("ymax").text)
            xmax = int(box.find("xmax").text)
        
        #Formata e adiciona a lista um conjunto de coordenadas correspondente a uma célula anotada
        list_with_single_boxes = [xmin, ymin, xmax, ymax]
        list_with_all_boxes.append(list_with_single_boxes)
        list_with_all_centers.append(coord_to_center(xmin, ymin, xmax, ymax))

    return filename, list_with_all_boxes, list_with_all_centers

### - Funções para a extração das métricas de avaliação do projeto

In [5]:
def minDist(element, list_of_elements):
    '''
    point, list of points -> float
    Dadas as coordenadas do centro de um circulo, e uma lista com coordenadas de centros de outros círculos em 
    um mesmo frame, retorna a menor distância entre este ponto central do circulo e os centros na lista
    Parâmetros
    ----------
    element: coordendas do centro do circulo
    list_of_elements: lista contendo os as coordenadas do centro de outros círculos.
    
    Return
    ------
    min(dist): distancia minima entre o circulo dado e os outros da lista.
    '''
    dist = []
    for element_of_list in list_of_elements:
        dist.append(scipy.spatial.distance.euclidean((element[1][0],element[1][1]),(element_of_list[1][0],element_of_list[1][1])))
    return min(dist)

In [6]:
def metricas(TP, FP, FN):
    '''
    int, int, int -> float, float, float
    Calcula as métricas de avaliação do método, Precisão, Revocação e Medida-F1.
    
    
    Parâmetros
    ----------
    TP: True positives, detecções que de fato pertencem ao quadro.
    FP: False positives, detecções que não pertencem ao quadro, de fato.
    FN: False negatives, células anotadas que não foram detectadas.
    
    Return
    ------
    Precisão: # de Acertos / # de Acertos + # de Erros
    Revocação: # de Acertos / #de Acertos + # de Omissões
    Medida-F1: 2 / (1/Precisão) + (1/Revocação)
    '''
    precisao = Decimal(TP) / Decimal(TP+FP)
    revocacao = Decimal(TP) / Decimal(TP+FN)
    f1 = Decimal(2) / Decimal(Decimal(1)/Decimal(precisao) + Decimal(1)/Decimal(revocacao))
    
    
    return precisao, revocacao, f1

In [7]:
def TP_FP_FN(list_of_detections, list_of_anotations, n_frames):
    '''
    lista de detecções, lista de anotações, int-> int, inr, int
    Calcula o número de True Positives, False Positives e False Negatives
    
    Parâmetros
    ----------
    list_of_detections: Lista contendo a identificação dos quadros do vídeo e as coordenadas dos centros dos
    círculos detectados.
    list_of_anotations: Lista contendo a identificação dos quadros do vídeo e as coordenadas dos centros dos 
    leucócitos anotados.
    n_frames: Número de frames contidos no vídeo
    
    Return
    ------
    TP: True positives, detecções que de fato pertencem ao quadro.
    FP: False positives, detecções que não pertencem ao quadro, de fato.
    FN: False negatives, células anotadas que não foram detectadas.
    '''
    #Variáveis locais
    TP = 0
    FP = 0
    FN = 0   
    #Para cada quadro
    for i in range(0, n_frames):
        #Variáveis para debug
        TP_frame = 0
        FP_frame = 0
        FN_frame = 0
    
        #Separa-se em listas auxiliares as células detectadas e as células anotadas naquele quadro
        detected_circles = [circle for circle in list_of_detections if circle[0] == i]
        anotated_leukocytes = [anotation for anotation in list_of_anotations if anotation[0] == i]
        
        #Extraí-se o número de celulas detectadas e o número de células anotadas naquele frame
        n_circles = len(detected_circles)
        n_leukocytes = len(anotated_leukocytes)
        
        #Para cada célula detectada, verifica se existe uma célula anotada em uma vizinhança de 7 pixels.
        for detected_circle in detected_circles:
            if minDist(detected_circle, anotated_leukocytes) <=7:
                TP+=1
                TP_frame+=1
            else:
                FP+=1
                FP_frame+=1
        #Para cada célula anotada, verifica se existe uma célula detectada em uma vizinhança de 7 pixels
        for leukocyte in anotated_leukocytes:
            if minDist(leukocyte, detected_circles) > 7:
                FN+=1
                FN_frame+=1       
        #print ("Frame {}: True positives = {} || False positives = {} || False negatives = {}.".format(file_index,TP_frame, FP_frame, FN_frame)) #Debug
        
    return TP,FP,FN

# Pipeline

###  - Aplicação da Transformada de Hough para Circulos em um conjunto de imagens de microscopia intravital capturado de veias do cérebro de uma cobaia.

In [8]:
#Definição do diretório fonte, contendo as imagens a serem processadas
diretorio = "to_process/"

#Definição da máscara que será aplicada sobre as imagens, segmentando a região de interesse
mask = cv2.imread('maskt1.png',0)

#Lista todos os arquivos no diretório
files = os.listdir(diretorio)    

#Organiza todos os arquivos encontrados
files = np.sort(files)

#Inicializa a lista que armazenará todos os círculos detectados
list_of_detections = [] 

#Extrai, uma-a-uma, as imagens do diretório para serem processadas 
num_files = len(files)
img_matrix = np.zeros((num_files,420,592), dtype=np.uint8)

for file_index, file in enumerate(files):
    img_matrix[file_index] = cv2.imread((diretorio+file),0)

    #Conversão para RGB, para sobrepor os círculos encontrados
    cimg = cv2.cvtColor(img_matrix[file_index],cv2.COLOR_GRAY2RGB)
    
    #Aplicação do unsharp mask para o destaque de bordas
    img_enhanced = unsharp_mask(img_matrix[file_index], 1)
   
    #Aplicação da máscara
    img_enhanced_masked = np.multiply(img_enhanced,mask)
    
    #Conversão para uint8, requisito do método
    img_enhanced_masked_uint8 = np.uint8(img_enhanced_masked)
    
    #Definição dos parâmetros 
    par_dp = 1        #The inverse ratio of resolution || Resolução da matriz acumuladora 
    par_min_dist = 8  #Minimum distance between detected centers || Mínima distância entre os circulos detectados
    par_param1 = 70   #Upper threshold for the internal Canny edge detector || Limite superior para o detector de bordas interno 
    par_param2 = 8    #Threshold for center detection. || Rigidez do votador responsável pelo julgamento dos candidatos a círculos detectados
    par_minRadius = 4 #mMinimum radio to be detected. If unknown, put zero as default. || Raio de busca mínimo
    par_maxRadius = 9 #Maximum radius to be detected. If unknown, put zero as default || Raio de busca máximo

    #Aplicação da Técnica
    circles = cv2.HoughCircles(img_enhanced_masked_uint8, #Input image (grayscale, uint8)
                           cv2.HOUGH_GRADIENT,            #CV_HOUGH_GRADIENT: Define the detection method. Currently this is the only one available in OpenCV
                           par_dp,par_min_dist,param1=par_param1,param2=par_param2,minRadius=par_minRadius,maxRadius=par_maxRadius)
    #A vector that stores sets of 3 values: x_{c}, y_{c}, r for each detected circle.
    
    if (len(circles) > 0):
        #Inseração dos circulos detectados sobre as imagens 
        circles = np.uint16(np.around(circles))
        for i in circles[0,:]:
            # draw the outer circle
            cv2.circle(cimg,(i[0],i[1]),i[2],(0,255,0),2)
            # draw the center of the circle
            cv2.circle(cimg,(i[0],i[1]),1,(0,0,255),1)
            list_of_detections.append((file_index, i[0:2]))
    else: 
        list_of_detections.append((file_index, -1))
    
    #Salvando um plot com o resultado da aplicação da técnica em um diretório
    #Subplot 1: 
    plt.figure(figsize=[30, 30])
    plt.subplot(1,3,1)
    plt.imshow(img_matrix[file_index],"gray")
    plt.xlabel("Imagem Original")
   
    #Subplot 2:
    plt.subplot(1,3,2)
    plt.imshow(cimg,"gray")
    plt.xlabel("Resultado @ dp = {}, min_dist = {}, tresh. edges  = {}, tresh. center = {}, minRad = {}, maxRad = {} c/ Mask e Unsharp Mask".format(par_dp, par_min_dist, par_param1, par_param2, par_minRadius, par_maxRadius))
    
    #Subplot 3: 
    ground_truth = cv2.imread("ground_truth_pipeline/img{}.jpg".format(file_index),0)
    plt.subplot(1,3,3)
    plt.imshow(ground_truth, "gray")
    plt.xlabel("Ground Truth")
    
    plt.savefig("processed_inspection/img{}.jpg".format(file_index),bbox_inches='tight')
    plt.close()
    
    #Salvando as imagens pré-processadas em suas dimensões originais em um outro diretório
    cv2.imwrite("processed_solo/img{}.jpg".format(file_index),img_matrix[file_index])
    
    #print file_index #Debug

### - Extração dos Ground Truths através das anotações presentes nos arquivos XML

In [9]:
#Definição dos diretórios fonte
diretorio_img = "processed_solo//"
diretorio_xml = "annotations_to_process/"

#Lista todos os arquivos no diretório
files_img = os.listdir(diretorio_img)   
files_xml = os.listdir(diretorio_xml)

#Organiza todos os arquivos encontrados
files_img = np.sort(files_img)
files_xml = np.sort(files_xml)

#Inicializa a lista que armazenará todos os leucócitos anotados
list_of_anotations = []

#Extrai, uma-a-uma, as imagens do diretório para serem processadas 
num_files_img = len(files_img)
img_matrix = np.zeros((num_files_img,420,592), dtype=np.uint8)

for file_index, file in enumerate(files_img):
    img_matrix[file_index] = cv2.imread((diretorio_img+file),0)

    #Conversão para RGB, para sobrepor os círculos encontrados
    cimg = cv2.cvtColor(img_matrix[file_index],cv2.COLOR_GRAY2RGB)
    
    #Extrai, para o frame atual, as anotações no arquivo XML correspondente
    name, boxes, center_of_boxes = read_content("annotations_to_process/brain_1__{}.xml".format(file_index+1))
   
    #Para cada célula anotada, desenha sobre a imagem uma caixa referente à anotação e um ponto em seu centro
    for cell in boxes:
        cv2.rectangle(cimg, (cell[0],cell[1]), (cell[2],cell[3]), (0,255,0), 1)
        cv2.circle(cimg,coord_to_center(cell[0],cell[1],cell[2],cell[3]), 1, (255,0,0), -1)
        
        #Adiciona na lista o índice do arquivo referente à celula anotada e as coordenadas do seu centro
        list_of_anotations.append((file_index, np.asarray(coord_to_center(cell[0],cell[1],cell[2],cell[3]))))
    
    #Salva o resultado
    cv2.imwrite("ground_truth_pipeline/img{}.jpg".format(file_index), cimg)      

### - Calculo dos resultados obtidos através da aplicação do método implementado

In [10]:
#Extraindo Métricas
TP, FP, FN = TP_FP_FN(list_of_detections, list_of_anotations, 220)

#Exibindo resultados
print "True Positives: {}, False Positives: {}, False Negatives: {}".format(TP,FP,FN)
print "Precisão: {:.2f}%, Revocação: {:.2f}%, Medida F1 = {:.2f}%.".format(metricas(TP,FP,FN)[0]*100,metricas(TP,FP,FN)[1]*100,metricas(TP,FP,FN)[2]*100)

True Positives: 2662, False Positives: 2276, False Negatives: 2929
Precisão: 53.91%, Revocação: 47.61%, Medida F1 = 50.57%.
